# TASK 3 , MAKE THE PIPELINE INPUT ANY lANGUAGE AND THEN TRANSLATE , ALSO TRY IMPLEMENTING FEW SHOT ALGORITHMS

In [1]:
%pip install sounddevice scipy numpy
%pip install requests pandas pydub pyaudio 
%pip install simpleaudio
%pip install dotenv soundfile
%pip install playsound
%pip install sarvamai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wave
import base64
import requests
import sounddevice as sd
import numpy as np
import time
import os
import io
from pydub import AudioSegment

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("API_KEY")

# INPUT OF THE AUDIO

In [4]:
import sounddevice as sd
import soundfile as sf
import threading

def record_audio(output_file='input_audio.wav', fs=44100):
    """Continuously record audio until user presses Enter, then save to a WAV file."""
    print("Recording... Press Enter to stop.")

    recording = []
    stop_recording = threading.Event()

    def wait_for_input():
        input()
        stop_recording.set()

    threading.Thread(target=wait_for_input).start()

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
        while not stop_recording.is_set():
            audio_chunk, _ = stream.read(1024)
            recording.append(audio_chunk)

    print("Recording stopped.")

    audio_data = np.concatenate(recording, axis=0)
    with wave.open(output_file, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(audio_data)

    print(f"Audio saved to {output_file}")
    return output_file



# USING STT TRANSLATE API INSTEAD OF THE STANDARD STT

The STT translate api is helpful in translating any language given in the input side , it automatically detects the language and then gives the transcribe in english.


This will later be passed on to a translation api for our required output

In [5]:
def split_audio(audio_path, chunk_duration_ms):
    audio = AudioSegment.from_file(audio_path) 
    chunks = []
    if len(audio) > chunk_duration_ms:
        for i in range(0, len(audio), chunk_duration_ms):
            chunks.append(audio[i : i + chunk_duration_ms])
    else:
        chunks.append(audio)
    return chunks
    
    
def translate_audio(audio_file_path,chunk_duration_ms= 30* 1000):
    
    api_url = "https://api.sarvam.ai/speech-to-text-translate"

    headers = {"api-subscription-key": api_key}  
    
    data = {
        "model": "saaras:v2",  
        "with_diarization": False,  
    }
    chunks = split_audio(audio_file_path, chunk_duration_ms)
    responses = []

    for idx, chunk in enumerate(chunks):
        chunk_buffer = io.BytesIO()
        chunk.export(chunk_buffer, format="wav")
        chunk_buffer.seek(0)  

        files = {"file": ("audiofile.wav", chunk_buffer, "audio/wav")}

        try:
            response = requests.post(api_url, headers=headers, files=files, data=data)
            if response.status_code == 200 or response.status_code == 201:
                print(f"Chunk {idx} POST Request Successful!")
                response_data = response.json()
                transcript = response_data.get("transcript", "")
                responses.append({"transcript": transcript})
            else:
                print(
                    f"Chunk {idx} POST Request failed with status code: {response.status_code}"
                )
                print("Response:", response.text)
        except Exception as e:
            print(f"Error processing chunk {idx}: {e}")
        finally:
            chunk_buffer.close()

    transcript = " ".join([resp["transcript"] for resp in responses])
    response = {
        "transcript": transcript,
        "language": response_data.get("language_code"),
    }
    return response
    

In [6]:
response=translate_audio("harvard.wav")

Chunk 0 POST Request Successful!


In [7]:
response=response.get("transcript")

# TRANSLATION MODULE

In [8]:

def translation(response,code):
    from sarvamai import SarvamAI
   
    client = SarvamAI(
        api_subscription_key=api_key
    )
    
    response = client.text.translate(
        input=response,
        source_language_code="en-IN",
        target_language_code=code,
        speaker_gender="Male"
    )
    return (response)

In [9]:
response=translation(response,"kn-IN")

In [10]:
response.translated_text

'ಹಳೆಯ ಬಿಯರ್\u200cನ ಹಳಸಿದ ವಾಸನೆ ಉಳಿದುಕೊಳ್ಳುತ್ತದೆ. ವಾಸನೆಯನ್ನು ಹೊರಹಾಕಲು ಬಿಸಿಲು ಬೇಕಾಗುತ್ತದೆ. ತಂಪಾದ ಸ್ನಾನವು ಆರೋಗ್ಯವನ್ನು ವಾಪಸ್ ತರುತ್ತದೆ. ಉಪ್ಪಿನ ಉಪ್ಪಿನಕಾಯಿ ಹಂದಿಮಾಂಸದೊಂದಿಗೆ ರುಚಿಯಾಗಿರುತ್ತದೆ. ಟ್ಯಾಕೋಸ್ ಅಲ್ ಪಾಸ್ಟರ್ ನನ್ನ ನೆಚ್ಚಿನದು. ಖಾರಭರಿತ ಆಹಾರವು ಹಾಟ್ ಕ್ರಾಸ್ ಬನ್ ಆಗಿದೆ.'

# TEXT TO SPEECH MODULE

In [11]:


def text_to_speech(transcript,code):
    api_url = "https://api.sarvam.ai/text-to-speech"
    
    headers = {
        "Content-Type": "application/json",
        "api-subscription-key": api_key
    }
    if code!="en-IN":
        payload = {
            "inputs": [transcript.translated_text],
            "target_language_code": code,
            "speaker": "abhilash",
            "model": "bulbul:v2",
            "pitch": 0,
            "pace": 1.0,
            "loudness": 1.0,
            "enable_preprocessing": True,
        }
        
    else:
        payload = {
            "inputs": [transcript],
            "target_language_code": code,
            "speaker": "abhilash",
            "model": "bulbul:v2",
            "pitch": 0,
            "pace": 1.0,
            "loudness": 1.0,
            "enable_preprocessing": True,
        }
    
    response = requests.post(api_url, json=payload, headers=headers)
    if response.status_code == 200:
        audio = response.json()["audios"][0]
        audio = base64.b64decode(audio)

        output_file = "output.wav"
        with wave.open(output_file, "wb") as wav_file:
            wav_file.setnchannels(1)  
            wav_file.setsampwidth(2)  
            wav_file.setframerate(22050)  
            wav_file.writeframes(audio)

        if os.path.exists(output_file):
            print(f"Audio file saved successfully as {output_file}")
        else:
            print(f"Failed to save audio file.")
    else:
        print(f"Error: {response.status_code}")
        print(response.json())

# MODULE TO PLAY THE AUDIO BACK

In [12]:
import threading

def play_audio(file_path):
    import soundfile as sf
    import sounddevice as sd
    data, fs = sf.read(file_path, dtype='float32')
    sd.play(data, fs)
    sd.wait()


In [14]:
code="en-IN"
record_audio("input.wav")
transcript = translate_audio("input.wav",chunk_duration_ms= 30 * 1000)
transcript = transcript.get("transcript")
if code=="en-IN":
    pass
else:
    transcript=translation(transcript,code)
text_to_speech(transcript,code)
play_audio("output.wav")

Recording... Press Enter to stop.
Recording stopped.
Audio saved to input.wav
Chunk 0 POST Request Successful!
Audio file saved successfully as output.wav
